In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import func, desc, distinct
from db import SessionLocal, Customers, Products, Sales

In [ ]:
session = SessionLocal()

In [ ]:
def get_total_sales_per_customer(session):
    """
    Calcula el total gastado por cada cliente.
    """
    query = (
        session.query(
            Customers.name.label("customer_name"),
            Customers.city,
            func.sum(Sales.quantity * Products.price).label("total_spent")
            
        )
        .join(Sales, Customers.customer_id == Sales.customer_id)
        .join(Products, Sales.product_id == Products.product_id)
        .group_by(Customers.name, Customers.city)
        .having(func.sum(Sales.quantity * Products.price) > 10000)
        .order_by(desc("total_spent"))
    )
    # Convertir el resultado de la consulta a un DataFrame de Pandas
    df = pd.read_sql(query.statement, session.bind)
    return df.head(10)

In [ ]:
df = get_total_sales_per_customer(session)

In [ ]:
if df.empty:
        print("No hay datos de productos para visualizar.")
        
plt.figure(figsize=(10, 6))
sns.barplot(x='total_spent', y='customer_name', hue = 'customer_name', data=df, palette='viridis')
plt.title('Top 5 Productos Más Vendidos por Cantidad')
plt.xlabel('Cantidad Total Vendida')
plt.ylabel('Producto')
plt.tight_layout()
plt.show()

In [ ]:
def get_top_selling_products(session, limit=5):
    """
    Encuentra los productos más vendidos por cantidad usando el ORM de SQLAlchemy.
    """
    query = (
        session.query(
            Products.name.label("product_name"),
            Products.category,
            func.sum(Sales.quantity).label("total_quantity_sold"),
        )
        .join(Sales, Products.product_id == Sales.product_id)
        .group_by(Products.name, Products.category)
        .order_by(desc("total_quantity_sold"))
        .limit(limit)
    )
    df = pd.read_sql(query.statement, session.bind)
    return df

In [ ]:
df =  get_top_selling_products(session)

## Usando seaborn

In [ ]:
if df.empty:
         print("No hay datos de productos para visualizar.")
plt.figure(figsize=(10, 6))
sns.barplot(x='total_quantity_sold', y='product_name', hue='product_name', data=df, palette='viridis')
plt.title('Top 5 Productos Más Vendidos por Cantidad')
plt.xlabel('Cantidad Total Vendida')
plt.ylabel('Producto')
plt.tight_layout()
plt.show()

In [ ]:
def get_monthly_sales_trend(session):
    """
    Calcula la tendencia de ventas mensuales usando el ORM de SQLAlchemy.
    """
    # La función to_char es específica de PostgreSQL, la usamos con func
    sales_month = func.to_char(Sales.sale_date, "YYYY-MM").label("sales_month")

    query = (
        session.query(
            sales_month,
            func.sum(Sales.quantity * Products.price).label("total_sales"),
        )
        .join(Products, Sales.product_id == Products.product_id)
        .filter(func.extract("year", Sales.sale_date) == 2025)
        .group_by(sales_month)
        .order_by(desc("total_sales"))
    )
    df = pd.read_sql(query.statement, session.bind)
    return df

In [ ]:
get_monthly_sales_trend(session)

In [ ]:
def get_total_sales_per_customer_gender(session):
    
    query = (
        session.query(
            Customers.gender.label("customer_gender"),
            func.count(distinct(Customers.customer_id)).label("total_sales"),
        )
        .join(Sales, Customers.customer_id == Sales.customer_id)
        .join(Products, Sales.product_id == Products.product_id)
        .group_by(Customers.gender)
        .order_by(desc("total_sales"))
    )
    # Convertir el resultado de la consulta a un DataFrame de Pandas
    df = pd.read_sql(query.statement, session.bind)
    return df

### Usando matplotlib

In [ ]:
df_customer_gender = get_total_sales_per_customer_gender(session)
plt.scatter(df_customer_gender['customer_gender'], df_customer_gender['total_sales'])
plt.show()

## Usando Dataframe method

In [ ]:
df_customer_gender.plot(kind='scatter', x='customer_gender', y='total_sales')
plt.show()

In [ ]:
def profitable_categories(session):

    query = (
        session.query(
            Products.category.label("Categorias"),
            func.sum(Sales.quantity * Products.price).label("Facturacion"),
            func.avg(Sales.quantity).label("Ventas por unidad")
        )
        .join(Sales, Products.product_id == Sales.product_id)
        .group_by(Products.category)
        .order_by("Facturacion")
    )
    df = pd.read_sql(query.statement, session.bind)
    
    df['Ventas por unidad'] = df['Ventas por unidad'].round(2)
    df['Facturacion'] = df['Facturacion'].apply(
        lambda x: f'{x:,.2f}'.replace(',', 'X').replace('.', ',').replace('X', '.'))
    
    return df

In [ ]:
plt.plot(profitable_categories(session)['Categorias'], profitable_categories(session)['Facturacion'])
plt.show()

In [ ]:
plt.scatter(profitable_categories(session)['Categorias'], profitable_categories(session)['Facturacion'])
plt.show()